# 1. Prueba para obtener mensajes desde GMAIL

In [1]:
# Importar la clase InstalledAppFlow de la biblioteca google_auth_oauthlib
from google_auth_oauthlib.flow import InstalledAppFlow
# Importar el módulo base64 para codificar y decodificar datos
import base64
# Para crear clientes de servicio para las APIs de Google
from googleapiclient.discovery import build  

# Función para manejar la autenticación de Gmail
def gmail_authenticate():
    # Definir el alcance (scope) para el acceso a Gmail, solo lectura en este caso
    SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

    # Configurar el flujo de autenticación con las credenciales de OAuth y el alcance definido
    flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
    # Iniciar un servidor local para manejar la autenticación y obtener las credenciales
    creds = flow.run_local_server(port=0)

    # Devolver las credenciales obtenidas después de completar la autenticación
    return creds

# Función para recuperar los últimos 'n_mails' correos electrónicos
def get_last_n_emails(service, n_mails):
    # Solicitar la lista de mensajes para el usuario autenticado, limitado por 'n_mails'
    results = service.users().messages().list(userId='me', maxResults=n_mails).execute()
    # Obtener los mensajes de la respuesta de la API
    messages = results.get('messages', [])
    total_msg = []  # Lista para almacenar los mensajes procesados

    # Iterar sobre cada mensaje en la lista de mensajes
    for message in messages:
        # Obtener los detalles completos del mensaje usando su ID
        msg = service.users().messages().get(userId='me', id=message['id']).execute()

        # Extraer el payload y las partes del mensaje
        payload = msg.get('payload', {})
        parts = payload.get('parts', [])
        body = ""  # Variable para almacenar el cuerpo del mensaje
        # Verificar si hay datos codificados en base64 en el cuerpo del mensaje
        if 'data' in payload['body']:
            # Decodificar los datos del cuerpo del mensaje
            body = base64.urlsafe_b64decode(payload['body']['data']).decode("utf-8")
        else:
            # Iterar sobre las partes del mensaje para encontrar y decodificar el texto
            for part in parts:
                if part['mimeType'] == 'text/plain':
                    body = base64.urlsafe_b64decode(part['body']['data']).decode("utf-8")
                    break
        
        # Agregar el cuerpo del mensaje decodificado al diccionario del mensaje
        msg["body"] = body

        # Añadir el mensaje procesado a la lista total_msg
        total_msg.append(msg)
    # Devolver la lista de mensajes procesados
    return total_msg

# Función para obtener un valor específico de los encabezados de un correo electrónico
def get_value_for_key(email_headers, key):
    # Iterar sobre cada encabezado en los encabezados del correo electrónico
    for header in email_headers:
        # Verificar si el nombre del encabezado coincide con la clave buscada
        if header['name'] == key:
            # Devolver el valor del encabezado si se encuentra la clave
            return header['value']
    # Devolver None si no se encuentra la clave
    return None


def get_mails(n_mails):
    # Construir el cliente de servicio de Gmail con las credenciales autenticadas
    service = build('gmail', 'v1', credentials=gmail_authenticate())
    # Llamar a la función para obtener los últimos 10 correos electrónicos y almacenar la respuesta en 'msg'
    msg = get_last_n_emails(service, n_mails=n_mails)

    # Devolver los datos del mensaje obtenido
    return msg


## 1.1. Función para obtener mensajes

In [2]:
msg = get_mails(n_mails=3)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=694915007051-7ln9qfjl9jrakq6dtql35s1slenkhrl7.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A63212%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=OpXN2iHUUI5GR0ZO34QbIJELvgmRMP&access_type=offline


## 1.2. Respuesta individual de mensaje como diccionario

In [3]:
msg[0]

{'id': '18d5d2d7488d3087',
 'threadId': '18d5d2d7488d3087',
 'labelIds': ['UNREAD', 'CATEGORY_UPDATES', 'INBOX'],
 'snippet': 'View jobs in Spain ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏',
 'payload': {'partId': '',
  'mimeType': 'multipart/alternative',
  'filename': '',
  'headers': [{'name': 'Delivered-To',
    'value': 'enriquemejiagamarra@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a05:7000:8601:b0:554:80b2:1688 with SMTP id bb1csp982030mab;        Tue, 30 Jan 2024 17:41:16 -0800 (PST)'},
   {'name': 'X-Google-Smtp-Source',
    'value': 'AGHT+IGN/tOS9hOe8et5/BF7832szdz3QAHRopTsmCBgpkSkQmMCg4U2WTY5S6lO+eKbWzmHnCU5'},
   {'name': 'X-Received',
    'value': 'by 2002:a17:906:708:b0:a35:deb0:cd10 with SMTP id y8-20020a170906070800b00a35deb0cd10mr73836ejb.64.1706665276724;        Tue, 30 Jan 2024 17:41:16 -0800 (PST)'},
   {'name': '

## 1.3. Obtener el asunto del correo

In [4]:
message_dict = dict(msg[0])
email_headers = message_dict['payload']['headers']
subject = get_value_for_key(email_headers, 'Subject')
subject

'29 new jobs for “machine learning”'

## 1.4. Obtener el contenido corto correo

In [5]:
content = message_dict['snippet']
content

'View jobs in Spain ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏'

## 1.5. Loop para hacer print a contenido y asunto de cada correo

In [6]:
for message in msg:
    message_dict = dict(message)

    email_headers = message_dict['payload']['headers']
    subject = get_value_for_key(email_headers, 'Subject')

    content = message_dict['snippet']
    print('###################')
    print(subject)
    print(content)

###################
29 new jobs for “machine learning”
View jobs in Spain ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏
###################
POST/CON 24 tickets are on sale! Save 50% as an early adopter
Join us at Postman&#39;s biggest API conference ever ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌
###################
😱 ¡Felicidades, Jose! Tienes hasta S/150 de regalo
Nos volvimos locos con este regalo ¡Hoy vas a ganar! ¡Hoy vas a ganar! Jose, con tu Tarjeta Visa BCP te llevas hasta S/150 de devolución por tus primeras compras (1) . Obténla HOY 100% online y


## 2. Uso de Modelo pre-entrenado HuggingFace

In [7]:
# https://huggingface.co/openai-community/roberta-base-openai-detector
from transformers import pipeline
pipe = pipeline("text-classification", model="roberta-base-openai-detector")
print(pipe("Hello world! Is this content AI-generated?"))

C:\Users\usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'label': 'Real', 'score': 0.8036580085754395}]
